In [1]:
import sys
import os
import argparse
import csv
import numpy as np

# from torch.utils.data import Dataset, DataLoader
sys.path.insert(1, '../')

import torch

import torch.nn.functional as F

from dataset import NeuronData_3,SuperNeuronData
import pandas as pd
from torch.utils.data import Sampler
from collections import defaultdict
import random
from tqdm import tqdm

from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [2]:
NAMES = ['DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']

In [3]:

dir='E:/DATA/crunch/tmp/preprocessed'
val_set= [NeuronData_3(emb_folder=dir,train=False, split =True,name_list= [NAMES[i]],encoder_mode=False) for i in range(len(NAMES))]
              

DC1 is not in list
Type: 0, group: train
DC5
626 torch.Size([626, 1024])
DC1 is not in list
Type: 0, group: train
UC1_I
639 torch.Size([639, 1024])
DC1 is not in list
Type: 0, group: train
UC1_NI
678 torch.Size([678, 1024])
DC1 is not in list
Type: 0, group: train
UC6_I
724 torch.Size([724, 1024])
DC1 is not in list
Type: 0, group: train
UC6_NI
671 torch.Size([671, 1024])
DC1 is not in list
Type: 0, group: train
UC7_I
664 torch.Size([664, 1024])
DC1 is not in list
Type: 0, group: train
UC9_I
665 torch.Size([665, 1024])


In [8]:
sample_0= val_set[2]
sample_1= val_set[4]

In [13]:
sample_0

Data(x=[16, 1024], edge_index=[192], emb_centroid=[1024], cluster_centroid=[2], cell_exps=[16, 460], centroid_exps=[1, 460], cell_num=16, cell_ids=[16], slide_id=0)

In [9]:
mean_squared_error(sample_0.cell_exps[2],sample_0.cell_exps[4])

0.12170427513764266

In [10]:
mean_squared_error(sample_0.cell_exps[2],sample_0.centroid_exps[0])

0.07716503300919676

In [18]:
mse_list=[]
for i in range(sample_0.cell_num):
    mse= mean_squared_error(sample_0.cell_exps[i],sample_0.centroid_exps[0])
    print(mse)
    mse_list.append(mse)
sum(mse_list)/len(mse_list)

0.06490988533916561
0.06647651116858691
0.07716503300919676
0.06654195372879905
0.05467562184615126
0.06509038468405925
0.09893344194983528
0.0517656690762687
0.1523694226700321
0.05167693120238086
0.06356325484451508
0.07497563303614667
0.032213458941794414
0.11126580696240199
0.05523363621247621
0.091054951695723


0.07361947477297082

In [4]:

for val in val_set:
    mse_list=[]
    for sample in val:
        for i in range(sample.cell_num):
            mse= mean_squared_error(sample.cell_exps[i],sample.centroid_exps[0])
            # print(mse)
            mse_list.append(mse)

    print(sum(mse_list)/len(mse_list))

0.0817374720494238
0.09542469113220899
0.08845021092998301
0.10462438971302283
0.08908681880745592
0.1027306454074696
0.09626991100516923


In [23]:
len(val_set)

4667

In [68]:
mean_squared_error(sample_0.centroid_exps[0],sample_1.centroid_exps[0])

0.12735696149696737

## Modify kmeans

In [117]:
cluster_path='E:/DATA/crunch/tmp/cluster'
group= 'train'
cluster_dir=f'{cluster_path}/{group}/cluster_data_split'
with open(f'{cluster_dir}/{NAMES[6]}_kmeans.pkl','rb') as f:
                cell_kmeans = pickle.load(f)
with open(f'{cluster_dir}/{NAMES[6]}_cells.pkl','rb') as f:
                cell_list_cluster = pickle.load(f)

E:\conda\envs\thitogene\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [84]:
cell_kmeans.cluster_centers_

array([[ 8839.23809524, 12069.71428571],
       [ 4182.        , 13002.65217391],
       [ 9127.58333333,  6193.66666667],
       ...,
       [11387.82352941, 13189.17647059],
       [ 9401.76923077,  8652.69230769],
       [ 8500.56666667,  9986.5       ]])

In [93]:
centroids = cell_list_cluster.groupby('cluster')[['x', 'y']].mean().sort_index().reset_index().to_numpy()
cell_kmeans.cluster_centers_=centroids[:,1:]
cell_kmeans.labels_=np.array([i for i in range(6500)])
cell_kmeans.cluster_centers_,cell_kmeans.labels_

(array([[12069.71428571,  8839.23809524],
        [13875.69565217,  3955.        ],
        [ 6235.81818182,  9916.63636364],
        ...,
        [ 9839.05882353,  8618.        ],
        [ 8596.48148148, 11886.18518519],
        [15164.6       ,  6501.4       ]]),
 array([   0,    1,    2, ..., 6497, 6498, 6499]))

In [96]:

cluster_locations = pd.DataFrame([{'x': c[1], 'y': c[2]} for c in centroids])

In [101]:
n_clusters=int(len(cluster_locations)/10)
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(cluster_locations[['x', 'y']])
cluster_locations['cluster'] = kmeans.labels_

E:\conda\envs\thitogene\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [102]:
kmeans.labels_

array([181, 587, 558, ..., 528, 232, 254])

In [118]:
cell_list_cluster

,x,y,cell_id,counts,cluster,train
0,15821,18226,48,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4840,1
1,16493,18092,51,"[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5635,1
2,15833,18213,56,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",4840,1
3,16432,18050,59,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3591,1
4,16442,18178,66,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5985,0
...,...,...,...,...,...,...
196932,3024,2159,326465,"[0, 0, 0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",5172,1
196933,3882,2713,326474,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4998,1
196934,3798,2670,326478,"[0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",4998,1
196935,3068,2317,326481,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5172,1


## Check attention scores

In [6]:
!cd

C:\DATA\Crunch\Neuron\notebook


In [71]:
attention_dir = f'../model_result/80_24_1024_1024/attention_scores.pkl'
edge_index_dir = f'../model_result/80_24_1024_1024/edge_index.pkl'
with open(attention_dir, 'rb') as f:
    attention_scores = pickle.load(f)
with open(edge_index_dir, 'rb') as f:
    edge_index = pickle.load(f)

In [79]:
len(edge_index[0][0][0])

7570

In [67]:
sample_index=0
# edge_index= edge_index[0]
sample_edge_indices = edge_index[sample_index]
sample_attention_scores = attention_scores[sample_index]

In [62]:
node_of_interest=57
neighbors = sample_edge_indices[1][sample_edge_indices[0] == node_of_interest]
node_attention_scores = sample_attention_scores[sample_edge_indices[0] == node_of_interest]
node_attention_scores = node_attention_scores.mean(axis=1)

# Identify the neighbor with the highest attention
highest_attention_idx = torch.argmax(node_attention_scores)
highest_attention_neighbor = neighbors[highest_attention_idx]
highest_attention_neighbor

IndexError: argmax(): Expected reduction dim to be specified for input.numel() == 0.

In [45]:
node_attention_scores,neighbors

(tensor([0.0424, 0.0399, 0.0618, 0.0539, 0.0332, 0.0280, 0.0399, 0.0539, 0.0332,
         0.0618, 0.0424, 0.0980]),
 tensor([81, 60, 76, 64, 75, 84, 60, 64, 75, 76, 81, 57]))

In [27]:
neighbors = sample_edge_indices[0][sample_edge_indices[0] == node_of_interest]
neighbors

tensor([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60])

In [ ]:
for edge in sample_edge_indices.T:
    if edge[0]==edge[1]:
        print(edge[1])

In [ ]:
edge_index[0]

TypeError: len() of a 0-d tensor

In [58]:
for edge in edge_index[0]:
    print(edge)

tensor([26, 51])
tensor([51, 26])
tensor([26, 27])
tensor([27, 26])
tensor([26, 34])
tensor([34, 26])
tensor([26, 55])
tensor([55, 26])
tensor([26, 52])
tensor([52, 26])
tensor([26, 44])
tensor([44, 26])
tensor([27, 55])
tensor([55, 27])
tensor([27, 51])
tensor([51, 27])
tensor([27, 52])
tensor([52, 27])
tensor([27, 26])
tensor([26, 27])
tensor([27, 34])
tensor([34, 27])
tensor([27, 37])
tensor([37, 27])
tensor([28, 29])
tensor([29, 28])
tensor([28, 41])
tensor([41, 28])
tensor([28, 45])
tensor([45, 28])
tensor([28, 36])
tensor([36, 28])
tensor([28, 55])
tensor([55, 28])
tensor([28, 50])
tensor([50, 28])
tensor([29, 28])
tensor([28, 29])
tensor([29, 41])
tensor([41, 29])
tensor([29, 45])
tensor([45, 29])
tensor([29, 50])
tensor([50, 29])
tensor([29, 47])
tensor([47, 29])
tensor([29, 36])
tensor([36, 29])
tensor([30, 48])
tensor([48, 30])
tensor([30, 42])
tensor([42, 30])
tensor([30, 47])
tensor([47, 30])
tensor([30, 50])
tensor([50, 30])
tensor([30, 45])
tensor([45, 30])
tensor([30, 31